In [ ]:
# Install necessary packages
%pip install pandas plotly nbformat

In [ ]:
# Import necessary libraries
import json
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd

hover_label_dict =dict(
    align='left',
    bgcolor="white",
    font_size=12,
    font_family="Arial"
)

# Load both JSON files
with open('profile_results.json', 'r') as file:
    profile_data = json.load(file)

with open('profiler_n_frames.json', 'r') as file:
    frame_data = json.load(file)

# Create DataFrame for frame data
frames_df = pd.DataFrame(frame_data['frames'])

# Create DataFrame for profile data
profiles_df = pd.DataFrame(profile_data['profiles'])

# Visualization 1: Profile Overview
fig1 = go.Figure()

# Add bars for min, avg, max times
for metric in ['minMs', 'averageMs', 'maxMs']:
    fig1.add_trace(go.Bar(
        name=metric.replace('Ms', ' Time'),
        x=profiles_df['name'],
        y=profiles_df[metric],
        text=profiles_df[metric].round(2),
        textposition='auto',
    ))

fig1.update_layout(
    title='Profile Timing Overview',
    barmode='group',
    xaxis_title='Function',
    yaxis_title='Time (ms)',
    yaxis_type='log',  # Use log scale for better visibility
    hoverlabel=hover_label_dict
)

# Visualization 2: Frame Times Distribution
fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=frames_df['frame_number'],
    y=frames_df['frame_time_ms'],
    mode='lines+markers',
    name='Frame Time',
    hovertemplate='Frame %{x}<br>Time: %{y:.2f}ms'
))

# Add horizontal lines for statistics
stats = frame_data['statistics']
for stat, color in [
    ('average_frame_time_ms', 'green'),
    ('min_frame_time_ms', 'blue'),
    ('max_frame_time_ms', 'red')
]:
    value = stats[stat]
    fig2.add_hline(
        y=value,
        line_dash="dash",
        line_color=color,
        annotation_text=f'{stat.replace("_frame_time_ms", "").title()}: {value:.2f}ms'
    )

fig2.update_layout(
    title='Frame Times Over Time',
    xaxis_title='Frame Number',
    yaxis_title='Frame Time (ms)',
    hoverlabel=hover_label_dict
)

# Visualization 3: FPS Statistics
fig3 = go.Figure()

fps_stats = {
    k: v for k, v in frame_data['statistics'].items() 
    if k.startswith('fps_')
}

fig3.add_trace(go.Bar(
    x=list(fps_stats.keys()),
    y=list(fps_stats.values()),
    text=[f'{v:.1f}' for v in fps_stats.values()],
    textposition='auto'
))

fig3.update_layout(
    title='FPS Statistics',
    xaxis_title='Metric',
    yaxis_title='FPS',
    yaxis_type='log',  # Use log scale since highest FPS can be very large
    hoverlabel=hover_label_dict
)

# Visualization 4: Box Plot of Frame Times
fig4 = go.Figure()

# Add box plot for frame times
fig4.add_trace(go.Box(
    y=frames_df['frame_time_ms'],
    name='Frame Times',
    boxpoints='all',  # show all points
    jitter=0.3,  # spread points horizontally
    pointpos=-1.8,  # offset points
    boxmean=True,  # show mean
    marker=dict(color='lightblue'),
    line=dict(color='blue'),
    hovertemplate='Time: %{y:.2f}ms<extra></extra>'
))

# Add markers for quartiles
quartiles = frames_df['frame_time_ms'].quantile([0.25, 0.5, 0.75])
for q, (label, color) in enumerate([('25th', 'purple'), ('Median', 'red'), ('75th', 'orange')]):
    fig4.add_hline(
        y=quartiles.iloc[q],
        line_dash="dash",
        line_color=color,
        annotation_text=f"{label}: {quartiles.iloc[q]:.2f}ms",
        annotation_position="right"
    )

fig4.update_layout(
    title='Frame Time Distribution',
    yaxis_title='Frame Time (ms)',
    showlegend=False,
    hovermode='y',
    hoverlabel=hover_label_dict
)

# Visualization 5: Call Count Analysis
fig5 = px.bar(
    profiles_df,
    x='name',
    y='calls',
    title='Function Call Counts',
    labels={'name': 'Function', 'calls': 'Number of Calls'},
    text='calls'
)

fig5.update_traces(
    textposition='auto',
    marker_line_width=1.5,
    marker_line_color='black'
)

fig5.update_layout(
    hoverlabel=hover_label_dict
)

# Display figures
print(f"Session: {profile_data['session']}")
print(f"Timestamp: {profile_data['timestamp']}\n")

fig1.show()
fig2.show()
fig3.show()
fig4.show()
fig5.show()
